<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/mri_resampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Resampling MRI:<br>
Modifying voxel spacing, origin, get direction of a MRI following a reference MRI<br>
Download public MRIs and my reference MRI:

In [1]:
import gdown

url = 'https://drive.google.com/uc?id=12VsmYrHN-a0OzVlwP8CxkuAK4zTfMaaC'
gdown.download(url,'pituitary_public.zip',quiet=True)
!unzip -q pituitary_public.zip


url = 'https://drive.google.com/uc?id=1QwB-DFkvDHR0kqVffLrqDGJ3xY-47yJ6'
gdown.download(url,'ref_mri.nii.gz',quiet=True)

'ref_mri.nii.gz'

Installing lib:

In [2]:
!pip -q install SimpleITK

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 7.4 MB/s eta 0:00:00


Resampling: Modifying voxel spacing, origin, get direction of a MRI following a reference MRI

In [3]:
import nibabel as nib
from glob import glob
import numpy as np
import os
import SimpleITK as sitk

def get_resampling(ref_mri, mri, path):
    mri.SetSpacing(ref_mri.GetSpacing())
    #  mri.SetDirection(ref_mri.GetDirection())
    mri.SetOrigin(ref_mri.GetOrigin())
    writer = sitk.ImageFileWriter()
    writer.SetFileName(path)
    writer.Execute(mri)


ref_mri = sitk.ReadImage('ref_mri.nii.gz')

#resampling mri
path_all = glob('pituitary_public/*/*T1_C.nii')
for path in path_all:
    print('mri to resample:', path)
    mri = sitk.ReadImage(path)
    path = path.replace('pituitary_public', 'pituitary_resampled')
    os.makedirs(os.path.dirname(path), exist_ok = True)
    path = path.replace(os.path.basename(path), os.path.basename(os.path.dirname(path))+'_t1c.nii.gz')
    get_resampling(ref_mri, mri, path)
    print('saved in:', path)

#resampling mask
path_all = glob('pituitary_public/*/mask.nii')
for path in path_all:
    print('mri to resample:', path)
    mri = sitk.ReadImage(path)
    path = path.replace('pituitary_public', 'pituitary_resampled')
    os.makedirs(os.path.dirname(path), exist_ok = True)
    path = path.replace(os.path.basename(path), os.path.basename(os.path.dirname(path))+'_mask.nii.gz')
    get_resampling(ref_mri, mri, path)
    print('saved in:', path)

mri to resample: pituitary_public/mri_005/COR_T1_C.nii
saved in: pituitary_resampled/mri_005/mri_005_t1c.nii.gz
mri to resample: pituitary_public/mri_007/COR_T1_C.nii
saved in: pituitary_resampled/mri_007/mri_007_t1c.nii.gz
mri to resample: pituitary_public/mri_009/COR_T1_C.nii
saved in: pituitary_resampled/mri_009/mri_009_t1c.nii.gz
mri to resample: pituitary_public/mri_006/COR_T1_C.nii
saved in: pituitary_resampled/mri_006/mri_006_t1c.nii.gz
mri to resample: pituitary_public/mri_001/COR_T1_C.nii
saved in: pituitary_resampled/mri_001/mri_001_t1c.nii.gz
mri to resample: pituitary_public/mri_011/COR_T1_C.nii
saved in: pituitary_resampled/mri_011/mri_011_t1c.nii.gz
mri to resample: pituitary_public/mri_013/COR_T1_C.nii
saved in: pituitary_resampled/mri_013/mri_013_t1c.nii.gz
mri to resample: pituitary_public/mri_004/COR_T1_C.nii
saved in: pituitary_resampled/mri_004/mri_004_t1c.nii.gz
mri to resample: pituitary_public/mri_010/COR_T1_C.nii
saved in: pituitary_resampled/mri_010/mri_010_t1c